In [1]:
import json

def get_credentials(filepath):
    with open(filepath, 'r') as f:
        credentials = json.load(f)
    return credentials

# Path to your credentials file
credentials_filepath = 'credentials.json'

# Retreieve credentials
credentials = get_credentials(credentials_filepath)

# Access your credentials
username = credentials['username']
password = credentials['password']

In [ ]:
# Step 1: Log-in w/ credentials [DONE]
# Step 2: Sold Prices
# Step 3: City (Vancouver)

In [77]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

browser = webdriver.Chrome()
options = webdriver.ChromeOptions()
url = 'https://www.zoocasa.com/'
sleep(2)

lst_of_listings_links = []
browser.get(url)

try:
    WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@class="style_choice__sLSfh"][position()=2]'))).click()
except:
    pass

def login_credentials(username, password):
    # login 
    # WebDriverWait(browser,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@class="style_nav-btn__AHq2s"]'))).click()
    # Email
    email_login = WebDriverWait(browser,5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-testid='loginRegistrationEmailTextField']")))
    email_login.send_keys(username)
    email_login.send_keys(Keys.ENTER)
    #Password
    pass_login = WebDriverWait(browser,5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-testid='passwordLoginModal']")))
    pass_login.send_keys(password)
    pass_login.send_keys(Keys.ENTER)

def retrieve_sold_listings(city, page):
    sold_url = 'https://zoocasa.com/{}-bc-real-estate/sold?page={}'.format(city, page)
    browser.get(sold_url)

    # If this "Sign in Required" shows up
    try:
        WebDriverWait(browser,5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[class='style_new-overlay-text__o7G6k']"))).click()
        login_credentials(username,password)

        try:
            WebDriverWait(browser,5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-testid='overlayAcceptTermsMsg']"))).click()
            WebDriverWait(browser,5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "[data-testid='termsOfUseAcceptButton_31017']"))).click()
        except:
            pass
    except:
        pass
    browser.refresh()
    # Get href
    listings = WebDriverWait(browser,5).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, "[class='style_street-address__TsB7y']")))

    for listing in listings:
        # Find the <a> tag within the <p> tag
        a_tag = WebDriverWait(listing,5).until(EC.presence_of_element_located((By.XPATH, "./a")))
    
        # Get the value of the href attribute
        href_value = a_tag.get_attribute("href")
        # print(href_value)
        lst_of_listings_links.append(href_value)

    sleep(5)

list_of_cities = ['richmond', 'burnaby', 'vancouver','north-vancouver', 'delta','surrey']

pages = 84
city = list_of_cities[0]

for page in range(1,pages):
    retrieve_sold_listings(city, page+1)
    print(page)
    
sleep(10)
browser.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


In [87]:
import pandas as pd
table = pd.DataFrame(lst_of_listings_links)

In [95]:
table = table.rename(columns= {0:"Listing link"})

In [96]:
table

,Listing link
0,https://www.zoocasa.com/richmond-bc-real-estat...
1,https://www.zoocasa.com/richmond-bc-real-estat...
2,https://www.zoocasa.com/richmond-bc-real-estat...
3,https://www.zoocasa.com/richmond-bc-real-estat...
4,https://www.zoocasa.com/richmond-bc-real-estat...
...,...
2223,https://www.zoocasa.com/richmond-bc-real-estat...
2224,https://www.zoocasa.com/richmond-bc-real-estat...
2225,https://www.zoocasa.com/richmond-bc-real-estat...
2226,https://www.zoocasa.com/richmond-bc-real-estat...


In [101]:
table['Listing link'].drop_duplicates()

0       https://www.zoocasa.com/richmond-bc-real-estat...
1       https://www.zoocasa.com/richmond-bc-real-estat...
2       https://www.zoocasa.com/richmond-bc-real-estat...
3       https://www.zoocasa.com/richmond-bc-real-estat...
4       https://www.zoocasa.com/richmond-bc-real-estat...
                              ...                        
2223    https://www.zoocasa.com/richmond-bc-real-estat...
2224    https://www.zoocasa.com/richmond-bc-real-estat...
2225    https://www.zoocasa.com/richmond-bc-real-estat...
2226    https://www.zoocasa.com/richmond-bc-real-estat...
2227    https://www.zoocasa.com/richmond-bc-real-estat...
Name: Listing link, Length: 2211, dtype: object

In [102]:
table.to_csv("richmond_sold_listings.csv")